In [9]:
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForCausalLM, AutoConfig

In [10]:
model_id = "/Users/vermaa/Desktop/runs/epoch_10"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
config.vision_config.model_type = "davit"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=config,
).to(device)
processor = AutoProcessor.from_pretrained(
    model_id, trust_remote_code=True, config=config
)
"""
model = AutoModelForCausalLM.from_pretrained("/Users/vermaa/Desktop/runs/epoch_10", local_files_only=True)
processor = AutoProcessor.from_pretrained("/Users/vermaa/Desktop/runs/epoch_10", local_files_only=True)
"""

prompt = "<OD> Extract attributes brand, variety, weight of the product from artwork image in a valid key-value pair JSON."

image = Image.open("~/Desktop/test-fl-1.png")

inputs = processor(text=prompt, images=image, return_tensors="pt")

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    do_sample=False,
    num_beams=3
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`